### Install Libraries

In [1]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers pillow requests

Note: you may need to restart the kernel to use updated packages.


### Single Image Input

In [7]:
import requests
from PIL import Image
from io import BytesIO
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import pipeline

# Initialize the processor and model for generating image descriptions
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Initialize an emotion analysis pipeline
emotion_classifier = pipeline("sentiment-analysis", model="j-hartmann/emotion-english-distilroberta-base")

def get_image_description(image_url):
    # Fetch the image content from the URL
    response = requests.get(image_url)
    
    if response.status_code == 200:
        try:
            image = Image.open(BytesIO(response.content)).convert("RGB")
        except Exception as e:
            raise UnidentifiedImageError(f"Failed to identify the image: {e}")
    else:
        raise Exception(f"Failed to fetch image. Status code: {response.status_code}")

    # Generate image description
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    description = processor.decode(out[0], skip_special_tokens=True)
    return description

def analyze_emotion(description):
    # Analyze emotion based on the description
    emotion_result = emotion_classifier(description)
    emotion = emotion_result[0]["label"]
    return emotion

def generate_artwork_description_and_emotion(image_url):
    description = get_image_description(image_url)
    emotion = analyze_emotion(description)
    return description, emotion

# Use a different image URL that is valid
image_url = "https://images.unsplash.com/photo-1506748686214-e9df14d4d9d0"

description, emotion = generate_artwork_description_and_emotion(image_url)

print(f"Image Description: {description}")
print(f"Emotional Tone: {emotion}")


Image Description: a person standing on a mountain at sunset
Emotional Tone: fear


### 3 Input Images

In [10]:
import requests
from PIL import Image
from io import BytesIO
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import pipeline

# Initialize the processor and model for generating image descriptions
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Initialize an emotion analysis pipeline
emotion_classifier = pipeline("sentiment-analysis", model="j-hartmann/emotion-english-distilroberta-base")

def get_image_description(image_url, max_new_tokens=50):
    # Fetch the image content from the URL
    response = requests.get(image_url)
    
    if response.status_code == 200:
        try:
            image = Image.open(BytesIO(response.content)).convert("RGB")
        except Exception as e:
            raise UnidentifiedImageError(f"Failed to identify the image: {e}")
    else:
        raise Exception(f"Failed to fetch image. Status code: {response.status_code}")

    # Generate image description
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs, max_new_tokens=max_new_tokens)
    description = processor.decode(out[0], skip_special_tokens=True)
    return description

def analyze_emotion(description):
    # Analyze emotion based on the description
    emotion_result = emotion_classifier(description)
    emotion = emotion_result[0]["label"]
    return emotion

def generate_artwork_description_and_emotion(image_url):
    description = get_image_description(image_url)
    emotion = analyze_emotion(description)
    return description, emotion

# Test the function with three valid image URLs
image_urls = [
    "https://images.pexels.com/photos/414612/pexels-photo-414612.jpeg",  # Mountain landscape
    "https://images.pexels.com/photos/104827/cat-pet-animal-domestic-104827.jpeg",  # Cat
    "https://images.pexels.com/photos/91227/pexels-photo-91227.jpeg"   # Night sky with stars
]

for i, url in enumerate(image_urls, 1):
    try:
        description, emotion = generate_artwork_description_and_emotion(url)
        print(f"Image {i} Description: {description}")
        print(f"Image {i} Emotional Tone: {emotion}\n")
    except Exception as e:
        print(f"Failed to process image {url}: {e}")

# Next Steps:
# 1. Refining the Description: If the generated description or emotion doesn't fully align with expectations,
#    you can fine-tune the model, adjust the max_new_tokens, or try different images.
# 2. Experiment with different max_new_tokens values to see how it affects the description length.
# 3. Explore other models or techniques for generating more detailed or varied descriptions.


Image 1 Description: a wooden dock sits on a lake at night
Image 1 Emotional Tone: fear

Image 2 Description: a cat with a white face and yellow eyes
Image 2 Emotional Tone: surprise

Image 3 Description: a man smiling while standing in front of a bush
Image 3 Emotional Tone: joy

